In [ ]:
#ローカルファイルの読み込み
import data_make as dm
import Input_form as Info 
from tqdm.notebook import tqdm
import openpyxl
import git
import os
import pickle

In [ ]:
#取得するデータの指定
repo_url, from_ver, to_ver, client_id, client_secret = Info.data_get()  #リポジトリのURL指定, 比較するブランチ, APIのid, KEY

In [ ]:
repo_path = '../repo/' + to_ver

if not os.path.exists(repo_path):
    os.mkdir(repo_path)
    print(repo_path + 'を作成しました.')

#すでに../repoにファイルが存在するならクローンしない
if os.listdir(repo_path) == []:
    #リポジトリのクローン
    git.Repo.clone_from(repo_url, repo_path, branch=to_ver) #--> 後のバージョンのリポジトリ
    print('クローン成功')
else:
    print('すでに../repoにはファイルが存在しています')

In [ ]:
#ファイルの行数とファイルの依存関係の取得
java_line = dm.get_java_line(repo_path, repo_url, from_ver, to_ver, client_id, client_secret)
#コミット情報の取得
java_commit = dm.get_java_commit(repo_url, from_ver, to_ver, client_id, client_secret,java_line)
#変更行数の割合を取得
prob_list = dm.get_prob_list(java_line, java_commit)
#依存関係の取得
import_dependencies, exception_dependencies, class_interface_dic, extends_list, interface_extends_list = dm.get_dependencies(repo_path)

In [ ]:
#ファイル名のリストの作成
file_list = []
for java_elem in java_line:
    file_list.append(java_elem[0])

#ファイル名をjava_lineのものに合わせる
new_import_dependencies = []
for i in tqdm(import_dependencies):
    for j in file_list:
        if i[1] in j:
            new_import_dependencies.append([i[0],j])
            
new_exception_dependencies = []
for i in tqdm(exception_dependencies):
    for j in file_list:
        if i[1] in j:
            new_exception_dependencies.append([i[0],j])

In [ ]:
#ファイル名をjava_lineのものに合わせる
for i in extends_list:
    for j in java_line:
        if j[0] in i[0]:
            i[0] = j[0]
        if j[0] in i[1]:
            i[1] = j[0]

In [ ]:
#ファイル名をjava_lineのものに合わせる
for i in interface_extends_list:
    for j in java_line:
        if j[0] in i[0]:
            i[0] = j[0]
        if j[0] in i[1]:
            i[1] = j[0]

In [ ]:
all_dependencies = new_import_dependencies + new_exception_dependencies + extends_list + interface_extends_list

In [ ]:
#ファイル名のリストの作成
file_list = []
for java_elem in java_line:
    file_list.append(java_elem[0])

#java_lineにない例外依存関係を取り除く
new_all_dependencies = []
for depend in all_dependencies:
    if depend[0] in file_list and depend[1] in file_list:
        new_all_dependencies.append(i)

#無効グラフの作成
undirected_module = []
for i in new_all_dependencies:
    undirected_module.append([i[1],i[0]])

java_module = new_all_dependencies + undirected_module
len(java_module)

In [ ]:
print(len(new_import_dependencies))
print(len(new_exception_dependencies))
print(len(extends_list))
print(len(interface_extends_list))

In [ ]:
len(java_module)

In [ ]:
with open('java_line.bin', 'wb') as f:
    pickle.dump(java_line, f)
with open('java_commit.bin', 'wb') as f:
    pickle.dump(java_commit, f)
with open('prob_list.bin', 'wb') as f:
    pickle.dump(prob_list, f)
with open('java_module.bin', 'wb') as f:
    pickle.dump(java_module, f)